In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\ranu1\Downloads\clean_data_after_eda.csv")
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0


In [6]:
price_df = pd.read_csv(r"C:\Users\ranu1\Downloads\price_data (1).csv")
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [7]:
price_df.isnull().sum()

id                    0
price_date            0
price_off_peak_var    0
price_peak_var        0
price_mid_peak_var    0
price_off_peak_fix    0
price_peak_fix        0
price_mid_peak_fix    0
dtype: int64

In [9]:
price_df['price_p1']=price_df['price_off_peak_var']+price_df['price_off_peak_fix']
price_df['price_p2']=price_df['price_peak_var']+price_df['price_peak_fix']
price_df['price_p3']=price_df['price_mid_peak_var']+price_df['price_mid_peak_fix']
price_df['pp12']=price_df['price_p2']-price_df['price_p1']
price_df['pp23']=price_df['price_p3']-price_df['price_p2']
price_df['pp13']=price_df['price_p3']-price_df['price_p1']

In [10]:
price_df.isnull().sum()

id                    0
price_date            0
price_off_peak_var    0
price_peak_var        0
price_mid_peak_var    0
price_off_peak_fix    0
price_peak_fix        0
price_mid_peak_fix    0
price_p1              0
price_p2              0
price_p3              0
pp12                  0
pp23                  0
pp13                  0
dtype: int64

In [11]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff_1 = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.rename(columns={'price_off_peak_var': 'jan_1', 'price_off_peak_fix': 'jan_2'}).drop(columns='price_date'), on='id')
diff_1['diff_dec_january_energy_p1'] = diff_1['dec_1'] - diff_1['jan_1']
diff_1['diff_dec_january_power_p1'] = diff_1['dec_2'] - diff_1['jan_2']
diff_1 = diff_1[['id', 'diff_dec_january_energy_p1','diff_dec_january_power_p1']]
diff_1.head()

,id,diff_dec_january_energy_p1,diff_dec_january_power_p1
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Difference between peak prices in December and preceding January

In [12]:
# Group peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_peak_var': 'mean', 'price_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff_2 = pd.merge(dec_prices.rename(columns={'price_peak_var': 'dec_1', 'price_peak_fix': 'dec_2'}), jan_prices.rename(columns={'price_peak_var': 'jan_1', 'price_peak_fix': 'jan_2'}).drop(columns='price_date'), on='id')
diff_2['diff_dec_january_energy_p2'] = diff_2['dec_1'] - diff_2['jan_1']
diff_2['diff_dec_january_power_p2'] = diff_2['dec_2'] - diff_2['jan_2']
diff_2 = diff_2[['id', 'diff_dec_january_energy_p2','diff_dec_january_power_p2']]
diff_2.head()

,id,diff_dec_january_energy_p2,diff_dec_january_power_p2
0,0002203ffbb812588b632b9e628cc38d,-0.002302,0.097749
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,-0.005120,0.097749
4,00114d74e963e47177db89bc70108537,0.000000,0.000000


Difference between mid-peak prices in December and preceding January

In [13]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_mid_peak_var': 'mean', 'price_mid_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff_3 = pd.merge(dec_prices.rename(columns={'price_mid_peak_var': 'dec_1', 'price_mid_peak_fix': 'dec_2'}), jan_prices.rename(columns={'price_mid_peak_var': 'jan_1', 'price_mid_peak_fix': 'jan_2'}).drop(columns='price_date'), on='id')
diff_3['diff_dec_january_energy_p3'] = diff_3['dec_1'] - diff_3['jan_1']
diff_3['diff_dec_january_power_p3'] = diff_3['dec_2'] - diff_3['jan_2']
diff_3 = diff_3[['id', 'diff_dec_january_energy_p3','diff_dec_january_power_p3']]
diff_3.head()

,id,diff_dec_january_energy_p3,diff_dec_january_power_p3
0,0002203ffbb812588b632b9e628cc38d,0.003487,0.065166
1,0004351ebdd665e6ee664792efc4fd13,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,0.000763,0.065166
4,00114d74e963e47177db89bc70108537,0.000000,0.000000


In [14]:
diff = pd.merge(pd.merge(diff_1, diff_2, on='id'), diff_3, on='id')
diff.head()

,id,diff_dec_january_energy_p1,diff_dec_january_power_p1,diff_dec_january_energy_p2,diff_dec_january_power_p2,diff_dec_january_energy_p3,diff_dec_january_power_p3
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916,-0.002302,0.097749,0.003487,0.065166
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779,0.000000,0.000000,0.000000,0.000000
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000,0.000000,0.000000,0.000000,0.000000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916,-0.005120,0.097749,0.000763,0.065166
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001,0.000000,0.000000,0.000000,0.000000


In [16]:
df['tenure'] = (df['date_end']-df['date_activ']).dt.days/365
df['tenure']

0        3.002740
1        7.030137
2        6.005479
3        6.005479
4        6.150685
           ...   
14601    3.958904
14602    4.002740
14603    4.000000
14604    4.002740
14605    7.002740
Name: tenure, Length: 14606, dtype: float64

### The deviation of last month consumption relative to the mean consumption for one month
This feature aims at detecting any change of pattern in the consumption of the client for the last month.

In [17]:
df['cons_dev']=(df['cons_12m']/12)-df['cons_last_month']
df['cons_dev'].describe()

count     14606.000000
mean      -2821.912564
std       21686.353629
min     -449656.000000
25%        -496.312500
50%         105.125000
75%         757.375000
max      194525.250000
Name: cons_dev, dtype: float64

In [18]:
df['cons_dev'].isnull().sum()

0

### Ratio of consumption for the next year compared to the current year
This feature shows whether the client is expected to have a larger or lower demand for energy and power in the future compared to his current consumption.

In [19]:
df['cons_pattern']=df['forecast_cons_12m']/df['cons_12m']
df['cons_pattern'].describe()

C:\Users\ranu1\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


count    1.450600e+04
mean              inf
std               NaN
min      0.000000e+00
25%      4.496907e-02
50%      1.024782e-01
75%      1.475232e-01
max               inf
Name: cons_pattern, dtype: float64

In [20]:
def handleInf(x):
    if x==float('-inf') or x==float('inf'):
        return 0
    else:
        return x

In [21]:
df.cons_pattern=df.cons_pattern.apply(handleInf)

In [22]:
df['cons_pattern'].describe()

count    14506.000000
mean         0.093761
std          0.054838
min          0.000000
25%          0.044747
50%          0.102414
75%          0.147496
max          0.624622
Name: cons_pattern, dtype: float64

In [23]:
df['cons_pattern'].isnull().sum()

100

There are a 100 missing values! let's see what all of that is about:

In [24]:
df[df['cons_pattern'].isnull()][['forecast_cons_12m', 'cons_12m', 'cons_pattern']]

,forecast_cons_12m,cons_12m,cons_pattern
0,0.0,0,NaN
135,0.0,0,NaN
268,0.0,0,NaN
344,0.0,0,NaN
392,0.0,0,NaN
...,...,...,...
13791,0.0,0,NaN
13899,0.0,0,NaN
13999,0.0,0,NaN
14144,0.0,0,NaN


In [25]:
df['cons_pattern'] = df['cons_pattern'].fillna(0)

In [26]:
df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,tenure,cons_dev,cons_pattern
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,2.086294,99.530517,44.235794,2.086425,9.953056e+01,4.423670e+01,1,3.002740,0.000000,0.000000
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000e+00,0,7.030137,388.333333,0.040762
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000e+00,0,6.005479,45.333333,0.088162
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000,0.000000,0.000000,0.000003,0.000000e+00,0.000000e+00,0,6.005479,132.000000,0.151540
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.000000,0.000000,0.000000,0.000011,2.896760e-06,4.860000e-10,0,6.150685,-157.250000,0.100734


In [27]:
train = pd.merge(price_df.drop(['price_date', 'price_off_peak_var', 'price_off_peak_fix', 'price_peak_var',
                                'price_peak_fix', 'price_mid_peak_var', 'price_mid_peak_fix'], axis=1), 
                 df.drop(['date_activ', 'date_end', 'date_modif_prod', 'date_renewal'], axis=1), on='id')
train.head()

,id,price_p1,price_p2,price_p3,pp12,pp23,pp13,channel_sales,cons_12m,cons_gas_12m,...,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn,tenure,cons_dev,cons_pattern
0,038af19179925da21a25619c5a24b745,44.418298,0.0,0.0,-44.418298,0.0,-44.418298,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0.0,0.0,0.0,0.000003,0.0,0.0,0,3.00274,-332.0,0.14863
1,038af19179925da21a25619c5a24b745,44.418298,0.0,0.0,-44.418298,0.0,-44.418298,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0.0,0.0,0.0,0.000003,0.0,0.0,0,3.00274,-332.0,0.14863
2,038af19179925da21a25619c5a24b745,44.418298,0.0,0.0,-44.418298,0.0,-44.418298,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0.0,0.0,0.0,0.000003,0.0,0.0,0,3.00274,-332.0,0.14863
3,038af19179925da21a25619c5a24b745,44.416557,0.0,0.0,-44.416557,0.0,-44.416557,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0.0,0.0,0.0,0.000003,0.0,0.0,0,3.00274,-332.0,0.14863
4,038af19179925da21a25619c5a24b745,44.416557,0.0,0.0,-44.416557,0.0,-44.416557,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0.0,0.0,0.0,0.000003,0.0,0.0,0,3.00274,-332.0,0.14863


In [28]:
train = pd.merge(train, diff, on='id')
train.head()

,id,price_p1,price_p2,price_p3,pp12,pp23,pp13,channel_sales,cons_12m,cons_gas_12m,...,churn,tenure,cons_dev,cons_pattern,diff_dec_january_energy_p1,diff_dec_january_power_p1,diff_dec_january_energy_p2,diff_dec_january_power_p2,diff_dec_january_energy_p3,diff_dec_january_power_p3
0,038af19179925da21a25619c5a24b745,44.418298,0.0,0.0,-44.418298,0.0,-44.418298,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0,3.00274,-332.0,0.14863,-0.005508,0.177779,0.0,0.0,0.0,0.0
1,038af19179925da21a25619c5a24b745,44.418298,0.0,0.0,-44.418298,0.0,-44.418298,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0,3.00274,-332.0,0.14863,-0.005508,0.177779,0.0,0.0,0.0,0.0
2,038af19179925da21a25619c5a24b745,44.418298,0.0,0.0,-44.418298,0.0,-44.418298,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0,3.00274,-332.0,0.14863,-0.005508,0.177779,0.0,0.0,0.0,0.0
3,038af19179925da21a25619c5a24b745,44.416557,0.0,0.0,-44.416557,0.0,-44.416557,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0,3.00274,-332.0,0.14863,-0.005508,0.177779,0.0,0.0,0.0,0.0
4,038af19179925da21a25619c5a24b745,44.416557,0.0,0.0,-44.416557,0.0,-44.416557,foosdfpfkusacimwkcsosbicdxkicaua,3576,0,...,0,3.00274,-332.0,0.14863,-0.005508,0.177779,0.0,0.0,0.0,0.0


In [29]:
#Modelling:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, f1_score, precision_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

In [30]:
train.drop_duplicates(inplace=True)
train.shape

(56369, 55)

In [31]:
y = train['churn']
X = train.drop(['churn'], axis=1).set_index('id')
X.dtypes

price_p1                          float64
price_p2                          float64
price_p3                          float64
pp12                              float64
pp23                              float64
pp13                              float64
channel_sales                      object
cons_12m                            int64
cons_gas_12m                        int64
cons_last_month                     int64
forecast_cons_12m                 float64
forecast_cons_year                  int64
forecast_discount_energy          float64
forecast_meter_rent_12m           float64
forecast_price_energy_off_peak    float64
forecast_price_energy_peak        float64
forecast_price_pow_off_peak       float64
has_gas                            object
imp_cons                          float64
margin_gross_pow_ele              float64
margin_net_pow_ele                float64
nb_prod_act                         int64
net_margin                        float64
num_years_antig                   

In [32]:
X['has_gas'] = X['has_gas'].map({'f': 0, 't': 1})

In [33]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6,24])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [35]:
X_train.shape, X_test.shape

((42276, 65), (14093, 65))

In [36]:
y_train.shape, y_test.shape

((42276,), (14093,))

In [37]:
def evaluate(model_, X_test_, y_test_):
    y_pred = model_.predict(X_test_)

    results = pd.DataFrame({"Accuracy"  : [accuracy_score(y_test_, y_pred)],
                            "Precision" : [precision_score(y_test_, y_pred)],
                            "Recall"    : [recall_score(y_test_, y_pred)],
                            "f1"        : [f1_score(y_test_, y_pred)],
                            "ROC AUC"   : [roc_auc_score(y_test, y_pred)]})
    return results

In [ ]:
parameters = {'n_estimators': [256, 512, 1024], 'max_depth': [6, 8, 12], 'learning_rate': [0.03, 0.1]}
model = xgb.XGBClassifier(n_jobs=-1, random_state=44, eval_metric="mlogloss")
gs = GridSearchCV(model, parameters, scoring='f1', cv=5)
gs.fit(X_train, y_train)

In [ ]:
pd.DataFrame(gs.cv_results_)

In [ ]:
gs.best_score_

In [ ]:
gs.best_params_